In [2]:
pip install pandas scikit-learn numpy

In [3]:
import pandas as pd


In [4]:
 #Mount Google Drive - applicable, if working on Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Set Working Directory - if working on Google Drive
%cd /content/drive/MyDrive/Spotify_content_based_rec


/content/drive/MyDrive/Spotify_content_based_rec


In [11]:
#Importing the dataset and choosing limited columns that I feel would best contribute to the model
df = pd.read_csv('spotify_data_12_20_2023.csv')
df = df.filter(['album_id','artists','track_id','track_name','acousticness','danceability','energy','tempo','speechiness'],axis=1)
print(df)

<ipython-input-11-94028391c7f9>:1: DtypeWarning: Columns (25,26,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('spotify_data_12_20_2023.csv')


                      album_id                         artists  \
0       4zH8bbEjLe0Gdbv2FVLjnK                          ['AB']   
1       4zH8bbEjLe0Gdbv2FVLjnK              ['AB', '22nd Jim']   
2       4zH8bbEjLe0Gdbv2FVLjnK                 ['AB', 'Veeze']   
3       4zH8bbEjLe0Gdbv2FVLjnK                          ['AB']   
4       4zH8bbEjLe0Gdbv2FVLjnK                          ['AB']   
...                        ...                             ...   
375136  1MPAXuTVL2Ej5x0JHiSPq8                ['Taylor Swift']   
375137  6fyR4wBPwLHKcRtxgd4sGh                ['Taylor Swift']   
375138  7mzrIsaAjnXihW3InKjlC3                ['Taylor Swift']   
375139  11gfxXxJPd3j6sdWUyEA5S  ['Taylor Swift', 'Ed Sheeran']   
375140  19O46pvAPG1Xg6fxAuSU5F                ['Taylor Swift']   

                      track_id  \
0       5JucnTkqh8uCZGggL1xxvv   
1       4VWrKNG3ssyZeboTgQ7Kj1   
2       6JsVrTLJPSOSpgJSnZyfSK   
3       7pZpw1xJWIavCUGqXPPriP   
4       0ylqwc3gcJdJotKd4SfMiu   
...

In [12]:
#basic data cleansing- Null removal
print(df[['danceability','acousticness','tempo','energy']].isnull().sum())
df = df.dropna(subset=['danceability','acousticness','tempo','energy'])

danceability    777
acousticness    777
tempo           777
energy          777
dtype: int64


In [13]:
#selecting the columns I want for feature selection
features = df[['danceability','acousticness','tempo','energy']]
print(features)

        danceability  acousticness    tempo  energy
0              0.622       0.03050   72.546   0.554
1              0.899       0.01120  101.581   0.701
2              0.782       0.08150   99.789   0.716
3              0.944       0.00413  105.010   0.582
4              0.527       0.01290  172.723   0.641
...              ...           ...      ...     ...
375136         0.547       0.00556  139.908   0.808
375137         0.374       0.80600  109.025   0.504
375138         0.589       0.00491  112.982   0.805
375139         0.581       0.12400   80.048   0.469
375140         0.654       0.06500  108.051   0.405

[374364 rows x 4 columns]


In [14]:
#scaling my features and the range of these values are very different from each other
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
print(scaled_features)

[[ 0.26030728 -0.98919525 -1.51081693  0.11050252]
 [ 1.7119707  -1.04294505 -0.55937351  0.66696411]
 [ 1.09881322 -0.84716209 -0.61809528  0.7237459 ]
 ...
 [ 0.08736543 -1.06046248 -0.1857759   1.06065122]
 [ 0.04544013 -0.72880112 -1.26498505 -0.21126099]
 [ 0.42800847 -0.89311399 -0.34735908 -0.45352998]]


In [20]:
#using cosine distance
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(scaled_features)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [21]:
print(scaled_features.shape)

(374364, 4)


In [24]:
## Get the feature values of the target song (e.g., first song)

target_song = scaled_features[0]
target_song = target_song.reshape(1,-1)

# Find the nearest neighbors
distances, indices = knn.kneighbors(target_song)
print("Indices of similar songs:", indices)
print("Distances to similar songs:", distances)

# Display the similar songs
print("Similar songs:")
for i, index in zip(indices[0], distances[0]):
  print(df.iloc[i])

Indices of similar songs: [[     0 154782  69050 195422 188383 138428 206301 195083 176913  50532]]
Distances to similar songs: [[2.22044605e-16 9.43746757e-05 5.56207069e-04 5.75323882e-04
  5.84149603e-04 7.56199139e-04 7.61575733e-04 7.69673119e-04
  7.81543000e-04 9.23492179e-04]]
Similar songs:
album_id        4zH8bbEjLe0Gdbv2FVLjnK
artists                         ['AB']
track_id        5JucnTkqh8uCZGggL1xxvv
track_name                       Intro
acousticness                    0.0305
danceability                     0.622
energy                           0.554
tempo                           72.546
speechiness                      0.414
Name: 0, dtype: object
album_id        60CU8E5B0QtSUJri6U1Gvm
artists             ['Johnny Chester']
track_id        0ZrHoXAGOUVHyunBJE1cNE
track_name              Roadside Dinah
acousticness                     0.157
danceability                     0.602
energy                           0.546
tempo                           89.365
speechiness  